In [ ]:
import pandas as pd

tweets_df = pd.read_csv("twcs_clean.csv")

In [ ]:
import numpy as np
from dill import load as dillload
from glob import glob

# we'll read the sentiments into twcs_clean
# we begin by creating a np.array "sentis" of
# nans having length same as twcs_clean
sentis = np.full(shape = (len(tweets_df),), 
                 fill_value = np.nan)

## read in the sentiment dill files
sdill_paths = glob("sentis/*.list.dill")

# keep count of how many you're reading
n_sentis_read_total = 0
    
for sdill_path in sdill_paths[:]:
    
    # figure out start iloc
    pre_str = "_sentis_"
    post_str = "_to_"
    start_iloc = int(sdill_path[
        sdill_path.index(pre_str) + len(pre_str) :
        sdill_path.index(post_str)
        ])
    
    # figure out end iloc
    pre_str = "_to_"
    post_str = ".list"
    end_iloc = int(sdill_path[
        sdill_path.index(pre_str) + len(pre_str) :
        sdill_path.index(post_str)
        ])
    
    n_sentis_expected = end_iloc - start_iloc + 1
    
    sentilist = []
    with open(sdill_path, "rb") as f:
        sentilist = dillload(f)
    
    # check length for integrity
    n_sentis = len(sentilist)
    n_sentis_read_total += n_sentis
    if n_sentis != n_sentis_expected:
        print("   MISMATCH on " + sdill_path + ":")
        print("      n_sentis_expected", n_sentis_expected)
        print("      n_sentis", len(sentilist))
        
    else:
        sentis[start_iloc : end_iloc + 1] = sentilist
        
# integrity check
n_sentis_saved = np.sum(np.isfinite(sentis))
if n_sentis_read_total != n_sentis_saved:
    print("   MISMATCH on n_sentis:")
    print("      n_sentis_read_total", n_sentis_read_total)
    print("      n_sentis_saved", n_sentis_saved)
        
else:
    tweets_df["sentiment"] = sentis
    
tweets_df[["text", "sentiment"]].head()

In [ ]:
# SLOW! save new tweets_df

tweets_df.to_csv("tweets_w_senti.csv", index = False)

In [ ]:
# testing random samples for integrity

from gcp_sentiment import GCPSentiment

get_sentence_sentiments = GCPSentiment(credentialspath =
    "/home/vagrant/google_service_account_credentials/" +
    "project_our-rock-218016.credentials.json"
    ).get_sentence_sentiments

get_sentence_sentiments(["@116827 @121927 @822888 @456692 Srs. Wells Fargo I have 39 month " +
                         "waitting for an answer for my checking account fraud an way 2 save" +
                         " reported the same day the email notice of insufficient funds, I "+
                         "immediately contact the security department, to formulate the claim "+
                         "and no one has deigned to contact me."])
# need to start billing for project for this to work